In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from entity_disambiguation.preprocessing import process_input

ModuleNotFoundError: No module named 'entity_disambiguation'

In [13]:
!ls ..

LICENSE               embeddings            pyproject.toml
README.md             entity_disambiguation pytest.ini
README.rst            notebooks             submissions
data                  poetry.lock           tests


### Use preprocessing function to load training and testing data

In [2]:
#first specify the directory that the ACY dataset lives in on your local machine
mydir = '../../aida-conll-yago-dataset/'
#you could specify your own train-test-split percentage, here the default is 80%
train_x, train_y, test_x, test_y = process_input(mydir, train = 0.8)

In [3]:
#Observe data
train_x.head()

,token,in_between_word_count
0,german,2
1,british,3
2,brussels,6
3,european,3
4,commission,0


In [4]:
print(train_x.shape)
print(train_y.shape)

(17805, 2)
(17805,)


In [5]:
print(test_x.shape)
print(test_y.shape)

(4452, 2)
(4452,)


### Use the baseline model

In [6]:
my_path = '../../kwnlp/'
article_df = pd.read_csv(os.path.join(my_path, 'article.csv'))

anchor_df = pd.read_csv(os.path.join(my_path, 'anchor_target_counts.csv'))

at_count_df = anchor_df.copy()
def text_normalizer(text):                              
    """Return text after stripping external whitespace and lower casing."""   
    return str(text).strip().lower()

at_count_df["normalized_anchor_text"] = at_count_df["anchor_text"].apply(text_normalizer)
at_count_df = at_count_df.loc[at_count_df['normalized_anchor_text'].str.len() > 0, :]

at_count_df = pd.merge(
    at_count_df,
    article_df,
    how="inner",
    left_on="target_page_id",
    right_on="page_id")
at_count_df = at_count_df.rename(columns={
    'title': 'target_page_title',
    'item_id': 'target_item_id',
    'views': 'target_page_views',
    'count': 'anchor_target_count',
    'page_title': 'target_page_title'})

at_count_df = at_count_df[[
    "normalized_anchor_text",
    "target_page_id",
    "target_item_id",
    "target_page_title",
    "target_page_views",
    "anchor_target_count"]]
at_count_df.head()

,normalized_anchor_text,target_page_id,target_item_id,target_page_title,target_page_views,anchor_target_count
0,united states,3434750,30,United_States,460156,152451
1,american,3434750,30,United_States,460156,65722
2,usa,3434750,30,United_States,460156,8559
3,u.s.,3434750,30,United_States,460156,7633
4,us,3434750,30,United_States,460156,5288


In [11]:
df_new = at_count_df.sort_values(['anchor_target_count'])

In [12]:
df_new.head()

,normalized_anchor_text,target_page_id,target_item_id,target_page_title,target_page_views,anchor_target_count
15269228,iasc,182786,1653031,IASC,130,1
6549421,fused,845000,167738,Plastic_welding,1602,1
6549420,solvent,845000,167738,Plastic_welding,1602,1
6549419,plastic welding adhesive,845000,167738,Plastic_welding,1602,1
6549418,weld,845000,167738,Plastic_welding,1602,1


In [13]:
print(len(df_new))

15269229


In [ ]:
df_new = df_new.drop_duplicates(subset = 'normalized_anchor_text', )